# Diesel Consumption Report Automation Main Documentation

Created by: Chan Xing Wei

Created On: 17/03/2022

Last Updated: 01/04/2022 08:50 AM

## To-Do


## Changelog
### Version 2.1 [01/04/2022 08:50 AM]
* support for real XLS importing if XML importing failed
* Ask user to move the source files into the Data backup folder and delete all source files after complete importing without unhandled failure
* Ask if the user wants to go back to main menu after completing import all data
* record any unhandled error down the error log, name of skipped file and timestamp for post-program debugging

### Version 2.0 [30/03/2022 01:32 AM]
This version has been updated with the following changes
* Minor fine tuning
* Menu to operate two different operations:
    * Run the program
    * Configuration of parameters (two layer: simple & advanced)
* All settings are contained into a dictionary for easy save and load configuration
* Splitted the menu and config codes from the algorithm codes
* skip any file that raise unhandled error

### Version 1.1 [22/03/2022 01:44 PM]
* Added progress bar for lines writen
* Enhanced results printing (showing successful imported files, empty files and skipped files)
* Edited docstrings

### Version 1.0 [22/03/2022 12:00 AM]
The first version of this automation script, includes all basic and necessary features to make this script usable

## Abstract
This document contains everything about this automation script.

This script program is strictly for internal use only, so hard-coded attributes will sometime be used. Some attributes like dieselyear, directory, filename can be changed.

The main purpose of this script is to automate the heavily repetitive copy-pasting, data processing and excel calculation stuffs.

Running this script should automatically import all .xls and ONLY .xls files in specified directory into an excel workbook in a specified sheet in another specified directory.

This script is not complete by itself, it contains the menu and configuration program only, and it has to import the diesel_consumption_report_automation_algomodule.py in order to run the actual automation program

## Outline
* Changelog
* Abstract
* Workflow
* Notes
* Imports
* Variables
* Functions
    * Debug
    * Define Messages
    * Save Config
    * General Functions
    * Main Menu
    * Config Menu
    * Advanced Config Menu
* Main

## Workflow

![Workflow](https://drive.google.com/uc?id=10XK9ExzokuV_CgP7rYiRjSjMSqfOOqn8&authuser=xingwei.chan%40gotruck.com&usp=drive_fs)

## Notes
* You must not open the main destination excel in the middle of running this script.
* You must have both read and write access to the import source folder
* A "config.json" will be created in the same folder of this program, you should not delete or modify it from outside of this program
* You should not change the sheet name "***Data***" of the main destination excel unless you know what you are doing.
* You should always make sure *NEVER* add any data into any cell below the most bottom cell with data (best practice is not to touch the entire ***Data*** sheet unless you certainly know what you are doing)
* You should not rename, move, delete, or add column in the ***Data*** sheet

## Imports

In [ ]:
import sys
import os
from os import system
import json
import diesel_consumption_report_automation_algomodule as dalgo

## Variables



### Default Config

|***Variables***|Description|Authority|
|-|-|-|
|***destpath***|The folder directory of where the main diesel report excel file is located, it can be changed but user has to make sure the integrity of the necessary files in the directory|Basic|
|***importpath***|The directory folder name of where the sorce files to be imported are located|Basic|
|***backuppath***|The directory folder name of where the files stored as backup after successful importing|Basic|
|***dieselyear***|This can be change so to enable this script to be used in any year|Basic|
|***mainexcel***|The destination excel file name where all the final data should be stored|Advanced|
|***mainexcelsheet***|The destination excel sheet within the excel file specified in ***mainexcel***|Advanced|
|***mixerlist***|A default list containing the first letter of truck number for concrete mixer truck type determination|Advanced|
|***pickuplist***|A default list containing the first letter of truck number for pick up truck type determination|Advanced|
|***tipperlist***|A default list containing the first letter of truck number for tipper truck type determination|Advanced|
|***wheelloaderlist***|A default list containing the first letter of truck number for wheel loader truck type determination|Advanced|
|***dispsplit1***|A parameter to control the completion progress bar split length (i.e. 50 means the progress bar will be splitted with a newline and display a multiplier of 50 for every 50 files imported)|Advanced|
|***dispsplit2***|A parameter to control the completion progress bar split length (i.e. 50 means the progress bar will be splitted with a newline and display a multiplier of 50 for every 50 files imported)|Advanced|
|***benchmarking***|A parameter to determine is the console required to print benchmarking info (granular elapsed time)|Advanced|
|***tmpcsv***|A temporary string stored in csv format|Internal Use|
|***importsuccess***|A counter counting how many successful importation|Internal Use|
|***importempty***|A counter counting how many empty files leading to skipped importation|Internal Use|
|***imported***|A list containing all successful imported filename|Internal Use|
|***emptyfile***|A list containing all failed imported filename due to file is empty|Internal Use|
|***nonxlscount***|A counter counting how many non XLS files in the import source folder|Internal Use|
|***nonxls***|A list containing all skipped filename due to the file extension is not .xls|Internal Use|
|***skippedforerrorcount***|A counter counting how many files that raised unexpected error in the import source folder|Internal Use|
|***skippedforerror***|A list containing all skipped filename that raised unexpected error|Internal Use|
|***bytewriten***|The size of content that had been successfully writen to the destination excel file, in bytes|Internal Use|
|***linewriten***|A counter for how many lines that had been successfully writen to the destination excel file|Internal Use|

In [ ]:
default_config ={
'destpath' : 'G:\\Shared drives\\[R] Admin\\[R] Diesel\\2022\\Weekly\\',
'importpath' : 'G:\\Shared drives\\[R] Admin\\[R] Diesel\\2022\\Weekly\\Import source files (DELETE AFTER USE)\\',
'backuppath' : 'G:\\Shared drives\\[R] Admin\\[R] Diesel\\2022\\Weekly\\Data\\',
'dieselyear' : "2022",
'mainexcel' : "",
'mainexcelsheet' : "Data",
'mixerlist' : ("W"),
'pickuplist' : ("G","Y"),
'tipperlist' : ("X"),
'wheelloaderlist' : ("L","H","9"),
'dispsplit1' : 50,
'dispsplit2' : 500,
'benchmarking' : False,}
default_config.update({'mainexcel' : f"Diesel Weekly {default_config['dieselyear']}.xlsx"})

### Importing Config

In [ ]:
if 'config.json' not in os.listdir():
    config = default_config
    config.update({'mainexcel' : f"Diesel Weekly {config['dieselyear']}.xlsx"})
    with open('config.json','wt',encoding='utf-8') as flw:
        json.dump(config,flw,indent=True)
        flw.close()
else:
    with open('config.json','rt',encoding='utf-8') as flr:
        config = json.load(flr)
        flr.close()

### Global Variables

In [ ]:
currentscreen = "main"
mainmenusel = ("1","2","x")
configsel = ("1","2","3","4","5","a","b","x")
configdict = {  "1":(config['destpath'],"destpath","directory","Input the directory of the destination path containing your main excel sheet, Example: C:\AAA\BBB\CCC\DDD"),
                      "2":(config['importpath'],"importpath","directory","Input the directory of the source path containing all files to be imported, Example: C:\AAA\BBB\CCC\DDD"),
                      "3":(config['backuppath'],"backuppath","directory","Input the directory of the source path to store the backup files, Example: C:\AAA\BBB\CCC\DDD"),
                      "4":(config['dieselyear'],"dieselyear",str,"Input the year number of the current year diesel data that you are working on (only necessary to change once in a new year)"),
                      "5":(config['mainexcelsheet'],"mainexcelsheet",str,"Input the sheet name of the sheet that you wish to import data to"),}
advconfigsel = ("1","2","3","4","5","6","7","8","b","x")
advconfigdict = { "1":(config['mainexcel'],"mainexcel",str,"Input the destination excel filename with extensionthat you are working with (not recommended to change unless you know what you are doing)"),
                          "2":(config['mixerlist'],"mixerlist",tuple,"Input the entire new list of mixer truck plate first character separated by comma (replacing the list, not adding to it)"),
                          "3":(config['pickuplist'],"pickuplist",tuple,"Input the entire new list of mixer truck plate first character separated by comma (replacing the list, not adding to it)"),
                          "4":(config['tipperlist'],"tipperlist",tuple,"Input the entire new list of mixer truck plate first character separated by comma (replacing the list, not adding to it)"),
                          "5":(config['wheelloaderlist'],"wheelloaderlist",tuple,"Input the entire new list of wheel loader plate first character separated by comma (replacing the list, not adding to it)"),
                          "6":(config['dispsplit1'],"dispsplit1",int,"Input the desired 1st progress bar length"),
                          "7":(config['dispsplit2'],"dispsplit2",int,"Input the desired 2nd progress bar length"),
                          "8":(config['benchmarking'],"benchmarking",bool,"Input 'True' or 'False' to enable benchmarking of importing time"),}
contsel = ("1","2","x")

## Functions

### Debug
This funciton is meant for debugging use ONLY, it should not to be used by the user.

In [ ]:
def debug():
    '''
    Prints all the configuration variables.

    FOR DEBUG USE ONLY.
    '''
    print("\nDebugging Info:")
    print(config['destpath'],type(config['destpath']))
    print(config['importpath'],type(config['importpath']))
    print(config['backuppath'],type(config['backuppath']))
    print(config['dieselyear'],type(config['dieselyear']))
    print(config['mainexcel'],type(config['mainexcel']))
    print(config['mainexcelsheet'],type(config['mainexcelsheet']))
    print(config['mixerlist'],type(config['mixerlist']))
    print(config['pickuplist'],type(config['pickuplist']))
    print(config['tipperlist'],type(config['tipperlist']))
    print(config['wheelloaderlist'],type(config['wheelloaderlist']))
    print(config['dispsplit1'],type(config['dispsplit1']))
    print(config['dispsplit2'],type(config['dispsplit2']))
    print(config['benchmarking'],type(config['benchmarking']))

### Define Messages
This funciton is to redefine the menu messages. It should be invoked everytime before a message is called to ensure all the latest config variables printed in the message are up-to-date.

In [ ]:
def define_msg():
    '''Definition of the messages to update with the real time variable values'''
    global welcome_msg
    welcome_msg =\
'''
Diesel Consumption Report Automation Script

Please select the action (Enter number only):
    1. Run the XLS import script
    2. Configuration
    X. Quit
'''

    global config_msg
    config_msg =\
f'''
Please select one parameter to configure (Enter number only):
    1. destpath ({config['destpath']})
    2. importpath ({config['importpath']})
    3. backuppath ({config['backuppath']})
    4. dieselyear ({config['dieselyear']})
    5. mainexcelsheet ({config['mainexcelsheet']})
    A. Advanced Configuration
    B. Back to main menu
    X. Quit
'''

    global advconfig_msg
    advconfig_msg =\
f'''
Please select one advanced parameter to configure (Enter number only):
    1. mainexcel ({config['mainexcel']})
    2. mixerlist ({config['mixerlist']})
    3. pickuplist ({config['pickuplist']})
    4. tipperlist ({config['tipperlist']})
    5. wheelloaderlist ({config['wheelloaderlist']})
    6. dispsplit1 ({config['dispsplit1']})
    7. dispsplit2 ({config['dispsplit2']})
    8. benchmarking ({config['benchmarking']})
    B. Back to config menu
    X. Quit
'''
    global cont_menu_msg
    cont_menu_msg =\
f'''
Operation finished. Do you want to continue?
    1. Return to Main Menu
    2. Go to Configuration
    X. Quit
'''

### Save Config
This funciton is to save the latest config settings to a JSON located in the local directory containing this main program file.

In [ ]:
def save_config():
    '''Save the latest config settings to a JSON file in default directory'''
    with open('config.json','wt',encoding='utf-8') as flw:
        json.dump(config,flw,indent=True)
        flw.close()

### General Functions
This funciton is to create different type of functions according to the argument passed to it. It uses the concept of closure of function to return another function.

The purpose is to contain all related and similar functions into one big chunk so they do not need to be declared separately and makes the codes out of this general function cleaner.

The returned function should be able to do the following:
* Validate userinput (if necessary)
* Set the variable in the config dictionary with the new value

Functions available:
* _strtype
* _inttype
* _booltype
* _tupletype
* _dirtype

In [ ]:
def genfunc(vartype):
    '''
    Takes in a vartype (variable datatype), return a function of validating that
    vartype and set new value to the specified variable.

    Supported vartype: str, int, bool, tuple, "directory"
    Note: "directory" vartype shall include the quotes

    Example:
    x = 'a'
    f = genfunc(str)
    f(x,'b')
    print(x)
    >>>b
    '''
    def _strtype(var,newval):
        print(newval)
        ans3 = input("strPlease check if your change is correct (Y/N): ")
        if ans3.lower() == 'y':
            if type(newval) != str:
                print("String ONLY!")
                if globals()['currentscreen'] == 'config': config_menu()
                elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
            else:
                config.update({var:newval})
                if var == "dieselyear": config.update({'mainexcel' : f"Diesel Weekly {config['dieselyear']}.xlsx"})
        else:
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
            

    def _inttype(var,newval):
        print(newval)
        ans3 = input("Please check if your change is correct (Y/N): ")
        if ans3.lower() == 'y':
            try:
                newval = int(newval)
            except ValueError:
                print("Integer ONLY!")
                if globals()['currentscreen'] == 'config': config_menu()
                elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
            else:
                config.update({var:newval})
        else:
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
            

    def _booltype(var,newval):
        print(newval)
        ans3 = input("Please check if your change is correct (Y/N): ")
        if ans3.lower() != 'y':
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
        elif newval.lower() not in ("true","false"):
            print("'True' or 'False' ONLY!")
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
        else:
            if newval.lower() == "true": newval = True
            elif newval.lower() == "false": newval = False
            config.update({var:newval})

    def _tupletype(var,newval):
        newval = tuple(newval.strip().split(','))
        print(newval)
        ans3 = input("Please check if your change is correct (Y/N): ")
        if ans3.lower() == 'y':
            config.update({var:newval})
        else:
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()

    def _dirtype(var,newval):
        print(newval)        
        ans3 = input("Please check if your change is correct (Y/N): ")
        if ans3.lower() == 'y':
            if "\\" not in newval:
                print("You can input directory only! Example: C:\AAA\BBB\CCC\DDD")
                if globals()['currentscreen'] == 'config': config_menu()
                elif globals()['currentscreen'] == 'advconfig': advconfig_menu()
            else:
                newval += "\\"
                config.update({var:newval})
        else:
            if globals()['currentscreen'] == 'config': config_menu()
            elif globals()['currentscreen'] == 'advconfig': advconfig_menu()

    if vartype == str: return _strtype
    elif vartype == int: return _inttype
    elif vartype == bool: return _booltype
    elif vartype == tuple: return _tupletype
    elif vartype == "directory": return _dirtype

### Main Menu
This funciton is contains the UI program of the main menu.

In [ ]:
def main_menu():
    '''This funciton is contains the UI program of the main menu.'''
    globals()['currentscreen'] = 'main'
    system('cls')
    define_msg()
    print(welcome_msg)
    while True:
        try:
            ans = input("Action: ")
            assert ans.lower() in mainmenusel
        except AssertionError:
            print(f"Option unavailable: Please enter {mainmenusel} ONLY\n")
            continue
        else:
            break

    if ans == "1":
        system('cls')
        ans2 = input(
f'''Please make sure that you have copied all files that you want to import into the source folder:
"{config['importpath']}"
Enter 'Y' when you are ready to start importing the files
Enter 'B' to return to main menu: ''')
        if ans2.lower() == 'y':
            # Run Main Program
            dalgo.automate_import_data(config)
            print(cont_menu_msg)
            while True:
                try:
                    ans3 = input("Action: ")
                    assert ans3.lower() in contsel
                except:
                    print(f"Option unavailable: Please enter {contsel} ONLY\n")
                    continue
                else:
                    break

            if ans3 == "1": main_menu()
            elif ans3 == "2": config_menu()
            elif ans3.lower() == "x": sys.exit()
                    
        else: main_menu()
    elif ans == "2": config_menu()
    elif ans.lower() == "x": sys.exit()

### Config Menu
This funciton is contains the UI program of the config menu.

In [ ]:
def config_menu():
    '''This funciton is contains the UI program of the config menu.'''
    globals()['currentscreen'] = 'config'
    system('cls')
    define_msg()
    print(config_msg)
    while True:
        try:
            ans = input("Action: ")
            assert ans.lower() in configsel
        except AssertionError:
            print(f"Option unavailable: Please enter {configsel} ONLY\n")
            continue
        else:
            break
    if ans.lower() == 'a':
        advconfig_menu()
    elif ans.lower() == 'b':
        main_menu()
    elif ans.lower() == 'x':
        sys.exit()
    else:
        print(f"Current Value: {configdict[ans][0]}")
        print(configdict[ans][3])
        nv = input("Please enter the new value: ")
        applyconfig = genfunc(configdict[ans][2])
        applyconfig(configdict[ans][1],nv)
        save_config()
        print("Successfully applied changes")
##        debug()
        config_menu()

### Advanced Config Menu
This funciton is contains the UI program of the advanced config menu.

In [ ]:
def advconfig_menu():
    '''This funciton is contains the UI program of the advanced config menu.'''
    globals()['currentscreen'] = 'advconfig'
    system('cls')
    define_msg()
    print(advconfig_msg)
    while True:
        try:
            ans = input("Action: ")
            assert ans.lower() in advconfigsel
        except AssertionError:
            print(f"Option unavailable: Please enter {advconfigsel} ONLY\n")
            continue
        else:
            break

    if ans.lower() == 'b':
        config_menu()
    elif ans.lower() == 'x':
        sys.exit()
    else:
        print(f"Current Value: {advconfigdict[ans][0]}")
        print(advconfigdict[ans][3])
        nv = input("Please enter the new value: ")
        applyconfig = genfunc(advconfigdict[ans][2])
        applyconfig(advconfigdict[ans][1],nv)
        save_config()
        print("Successfully applied changes")
##        debug()
        advconfig_menu()

## Main Program
This section is the main program of the entire automation script.

In [ ]:
if __name__ == "__main__":
    main_menu()